# Financial Data Collection & Preprocessing
This notebook guides you through collecting historical stock data, extracting technical indicators, preprocessing the data, and saving it for use in trading models.

In [1]:
# Uncomment and run this cell if you haven't installed the required libraries
# !pip install yfinance pandas numpy ta fredapi scikit-learn


In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from fredapi import Fred
import datetime


## 1. Download Historical Stock Data

In [3]:
# Download historical stock data for Apple (AAPL)
ticker = "AAPL"
start_date = "2018-01-01"
end_date = "2023-12-31"

df = yf.download(ticker, start=start_date, end=end_date)
df.reset_index(inplace=True)
df.head()


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Price,Date,Close,High,Low,Open,Volume
Ticker,,AAPL,AAPL,AAPL,AAPL,AAPL
0,2018-01-02,40.479839,40.489241,39.774861,39.986357,102223600
1,2018-01-03,40.472797,41.017982,40.409352,40.543296,118071600
2,2018-01-04,40.660778,40.764175,40.437536,40.545630,89738400
3,2018-01-05,41.123711,41.210657,40.665476,40.757123,94640000
4,2018-01-08,40.970982,41.267071,40.872282,40.970982,82271200


## 2. Compute Technical Indicators (MACD, RSI, Bollinger Bands)

In [4]:
# MACD
df['EMA_12'] = df['Close'].ewm(span=12, adjust=False).mean()
df['EMA_26'] = df['Close'].ewm(span=26, adjust=False).mean()
df['MACD'] = df['EMA_12'] - df['EMA_26']
df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()

# RSI
delta = df['Close'].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
rs = gain / loss
df['RSI'] = 100 - (100 / (1 + rs))

# Bollinger Bands
rolling_mean = df['Close'].rolling(window=20).mean()
rolling_std = df['Close'].rolling(window=20).std()
df['Bollinger_Upper'] = rolling_mean + (rolling_std * 2)
df['Bollinger_Lower'] = rolling_mean - (rolling_std * 2)

df.tail()


Price,Date,Close,High,Low,Open,Volume,EMA_12,EMA_26,MACD,MACD_Signal,RSI,Bollinger_Upper,Bollinger_Lower
Ticker,,AAPL,AAPL,AAPL,AAPL,AAPL,,,,,,,
1504,2023-12-22,192.444595,194.243791,191.818350,194.015153,37122800,193.575128,190.932981,2.642147,3.246320,59.246152,198.299967,186.725379
1505,2023-12-26,191.897873,192.732856,191.679185,192.454528,28919300,193.317089,191.004454,2.312634,3.059583,49.031989,198.182203,187.167200
1506,2023-12-27,191.997269,192.345186,189.949565,191.341219,48087700,193.114039,191.077996,2.036043,2.854875,52.291529,198.093522,187.529242
1507,2023-12-28,192.424713,193.498269,192.017156,192.981369,34049900,193.007989,191.177753,1.830236,2.649947,47.920493,197.852635,188.188618
1508,2023-12-29,191.380951,193.239786,190.585722,192.742770,42628800,192.757676,191.192805,1.564871,2.432932,40.185208,197.634820,188.662891


## 3. Download Macroeconomic Indicators (Interest Rates from FRED)

In [5]:
# Get interest rate data from FRED
# You need a FRED API key from https://fred.stlouisfed.org/
fred = Fred(api_key="3f32de14e98a5e87100a6e520161e3d1")

# Example: US Federal Funds Rate (FEDFUNDS)
interest_rates = fred.get_series("FEDFUNDS", observation_start=start_date, observation_end=end_date)
interest_rates = interest_rates.reset_index()
interest_rates.columns = ['Date', 'FedFundsRate']
interest_rates.head()


,Date,FedFundsRate
0,2018-01-01,1.41
1,2018-02-01,1.42
2,2018-03-01,1.51
3,2018-04-01,1.69
4,2018-05-01,1.70


## 4. Preprocess the Dataset

In [ ]:
#  Flatten MultiIndex columns to regular column names
if isinstance(df.columns, pd.MultiIndex):
    df.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df.columns]

if isinstance(interest_rates.columns, pd.MultiIndex):
    interest_rates.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in interest_rates.columns]

#  Print Columns to Debug
print("Updated DF Columns:", df.columns)
print("Updated Interest Rates Columns:", interest_rates.columns)

#  Automatically detect and rename the 'Date' column
for col in df.columns:
    if "date" in col.lower():
        df.rename(columns={col: "Date"}, inplace=True)
        break  # Stop after renaming the correct column

#  Verify that 'Date' now exists
print("Final DF Columns:", df.columns)
assert 'Date' in df.columns, "ERROR: 'Date' is still missing in df!"

#  Convert 'Date' to datetime
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
interest_rates['Date'] = pd.to_datetime(interest_rates['Date'], errors='coerce')

#  Drop invalid dates
df.dropna(subset=['Date'], inplace=True)
interest_rates.dropna(subset=['Date'], inplace=True)

#  Ensure Sorting Before Merging
df = df.sort_values('Date').reset_index(drop=True)
interest_rates = interest_rates.sort_values('Date').reset_index(drop=True)

#  Perform Safe Merge
df = pd.merge_asof(df, interest_rates, on='Date')

#  Final Step: Display Results
print("Final Data Preview:")
print(df.head())


Updated DF Columns: Index(['Date_', 'Close_AAPL', 'High_AAPL', 'Low_AAPL', 'Open_AAPL',
       'Volume_AAPL', 'EMA_12_', 'EMA_26_', 'MACD_', 'MACD_Signal_', 'RSI_',
       'Bollinger_Upper_', 'Bollinger_Lower_'],
      dtype='object')
Updated Interest Rates Columns: Index(['Date', 'FedFundsRate'], dtype='object')
Final DF Columns: Index(['Date', 'Close_AAPL', 'High_AAPL', 'Low_AAPL', 'Open_AAPL',
       'Volume_AAPL', 'EMA_12_', 'EMA_26_', 'MACD_', 'MACD_Signal_', 'RSI_',
       'Bollinger_Upper_', 'Bollinger_Lower_'],
      dtype='object')
Final Data Preview:
        Date  Close_AAPL  High_AAPL   Low_AAPL  Open_AAPL  Volume_AAPL  \
0 2018-01-02   40.479839  40.489241  39.774861  39.986357    102223600   
1 2018-01-03   40.472797  41.017982  40.409352  40.543296    118071600   
2 2018-01-04   40.660778  40.764175  40.437536  40.545630     89738400   
3 2018-01-05   41.123711  41.210657  40.665476  40.757123     94640000   
4 2018-01-08   40.970982  41.267071  40.872282  40.970982     8

## 5. Save Cleaned Dataset

In [7]:
# Save to CSV
df.to_csv("cleaned_stock_data.csv", index=False)
print("Dataset saved as cleaned_stock_data.csv")


Dataset saved as cleaned_stock_data.csv
